In [1]:
import pandas as pd
import numpy as np

In [2]:
# load in sv2 merged table
# should have columns ['chrom', 'svtype', 'pos', 'end', 'svlength']
# can have more columns
svs = pd.read_csv('output/merged.sv2.breakpoints_resolved.tsv', sep='\t')

In [3]:
# downloaded from https://gnomad.broadinstitute.org/downloads#v2-structural-variants
gnomad = pd.read_csv('gnomad_v2.1_sv.nonneuro.sites.bed.gz', sep='\t')

# create gnomad column for chromosome with chr
gnomad['chrom'] = gnomad['#chrom'].apply(lambda s: 'chr{}'.format(s))

# filter for only dups and dels
gnomad = gnomad[gnomad.svtype.isin(['DEL', 'DUP'])].copy()

# filter for pass only
gnomad = gnomad[gnomad.FILTER == 'PASS'].copy()

# set gnomad frequency to type float
gnomad['AF'] = gnomad.AF.astype(float)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,22,24,36,37,46,47,61,62,72,73,82,83,96,97,106,107,116,117,130,131,140,141,150,151,164,165,174,175,184,185,198,199,208,209,218,219,232,233) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# loop over every chrom and svtype
chromosomes = list(svs.chrom.unique())
svtypes = ['DUP', 'DEL']
svs['gnomad_freq_non_neuro'] = '.'

for chrom in chromosomes:
    for svtype in svtypes:
        # get all strctural variants in chromosome and svtype
        sub_svs = svs[(svs.chrom == chrom) & (svs.svtype == svtype)].copy()
        sub_gnomad = gnomad[(gnomad['chrom'] == chrom) & (gnomad.svtype == svtype)].copy()
        
        # reset index in sub_gnomad
        sub_gnomad = sub_gnomad.reset_index(drop=True)
        
        # create numpy structures for end, start, and svlength
        # (numpy is faster than pandas)
        chrom_ends = sub_gnomad['end'].to_numpy()
        chrom_starts = sub_gnomad['start'].to_numpy()
        chrom_svlengths = chrom_ends - chrom_starts

        # iterate over all structural variants in cohort
        # and compare against other structural variants gnomad
        for i, row in sub_svs.iterrows():
            start = row['start']
            end = row['end']
            length = row['svlength']

            min_end   = np.minimum(chrom_ends, end)
            max_start = np.maximum(chrom_starts, start)
            max_length = np.maximum(chrom_svlengths, length)

            overlap = (min_end - max_start) > .5 * max_length

            # in some cases the structural variant overlaps with more than one gnomad SV
            # in those cases get the sum of all SV that it overlaps with
            allele_frequency = sub_gnomad.loc[overlap]['AF'].sum()
            
            # save to table
            svs.at[i, 'gnomad_freq_non_neuro'] = allele_frequency

In [5]:
svs.to_csv('output/merged.sv2.gnomad_non_neuro.tsv', sep='\t', index=False)

In [6]:
svs

,sample,svtype,chrom,start,end,svlength,genes,gnomad_freq_non_neuro
0,SG172,DUP,chr1,845001,850200,5200,intergenic,0.000497
1,SG172,DUP,chr1,13178567,13187129,8563,"NM_001136561,LOC440563,exon_1/1",0
2,SG172,DUP,chr1,31070606,31070669,64,intergenic,0.327312
3,SG172,DUP,chr1,56996032,56996346,315,"NM_003713,PPAP2B,intron_1/5",0
4,SG172,DUP,chr1,59644931,59670360,25430,intergenic,0
...,...,...,...,...,...,...,...,...
422002,SG445,DEL,chrX,151982765,151982840,76,intergenic,0.764914
422003,SG445,DEL,chrX,152807507,152807579,73,"NM_001001344,ATP2B3,intron_1/19|NM_021949,ATP2...",0.233868
422004,SG445,DEL,chrX,154365486,154365579,94,intergenic,0.459274
422005,SG445,DEL,chrX,154771601,154785600,14000,"NM_018196,TMLHE,exon_1/7|NM_018196,TMLHE,intro...",0.0123515
